In [358]:
import pandas as pd
import numpy as np
import os 

In [359]:
df = pd.read_csv("news_summary.csv", encoding = "ISO-8859-1")
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [360]:
import langdetect

def detect_language(text):
    return langdetect.detect(text)

df = pd.read_csv("news_summary.csv", encoding = "ISO-8859-1")
print("Nombre d'inputs avant suppression des textes non anglais : " + str(len(df)))

# créer une liste vide qui contiendra les indices des lignes à supprimer
indexes_to_delete = []

# parcourir chaque ligne du dataframe
for index, row in df.iterrows():

    lang = detect_language(row['headlines'][0:200])
    # si la valeur de l'attribut 'text' commence par un 'm'
    if type(row["ctext"]) == float:
    # ajouter l'index de cette ligne à la liste des indices à supprimer
        indexes_to_delete.append(index)

# supprimer les lignes du dataframe en utilisant la liste des indices à supprimer
df.drop(indexes_to_delete, inplace=True)

print("Nombre d'inputs après suppression des textes non anglais : " + str(len(df)))

Nombre d'inputs avant suppression des textes non anglais : 4514
Nombre d'inputs après suppression des textes non anglais : 4396


### PREPROCESSING


In [361]:
# REMOVE PUNCTUATION
import string

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

df["ctext"] = df["ctext"].apply(lambda row : remove_punctuation(row))
df["ctext"] = df["ctext"].apply(lambda row: row.lower())

df["text"] = df["text"].apply(lambda row : remove_punctuation(row))
df["text"] = df["text"].apply(lambda row: row.lower())

In [362]:
# TOKENIZATION
import nltk
import nltk.corpus
from nltk.tokenize import word_tokenize

df["ctext"] = df["ctext"].apply(lambda x : word_tokenize(x))

df["text"] = df["text"].apply(lambda x : word_tokenize(x))

In [363]:
# REMOVE STOP WORDS
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
df["ctext"] = df["ctext"].apply(lambda x: [w for w in x if not w in stop_words])

df["text"] = df["text"].apply(lambda x: [w for w in x if not w in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leolamoureux/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [364]:
# LEMMATIZATION
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

df["ctext"] = df["ctext"].apply(lambda x: [WordNetLemmatizer().lemmatize(w) for w in x])

df["text"] = df["text"].apply(lambda x: [WordNetLemmatizer().lemmatize(w) for w in x])

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leolamoureux/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [375]:
df = df.rename(columns={"text": "summary"})
df = df.rename(columns={"ctext": "article"})
df.to_csv('preprocessed_df.csv', index=False)